In [3]:
import torch.nn as nn
import torch
import pdb

import sys, os
import random
import numpy as np
from shutil import copy
import matplotlib.pyplot as plt
from copy import deepcopy

from omegaconf import OmegaConf
import shutil
import pickle
import random
from tqdm import tqdm

In [4]:
# run_path = '/home/harishbabu/projects/PIPNet/runs/010-CUB-27-imgnet_OOD_cnext26_img=224_nprotos=20'
# run_path = '/home/harishbabu/projects/PIPNet/runs/031-CUB-18-imgnet_cnext26_img=224_nprotos=20_orth-on-rel'
# run_path = '/home/harishbabu/projects/PIPNet/runs/032-CUB-18-imgnet_cnext26_img=224_nprotos=20_orth-on-rel'

# run_path = '/home/harishbabu/projects/PIPNet/runs/035-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel'

# run_path = '/home/harishbabu/projects/PIPNet/runs/043-035_clone-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel'
# run_path = "/home/harishbabu/projects/PIPNet/runs/036-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel_uniformity"
# run_path = "/home/harishbabu/projects/PIPNet/runs/041-035_clone-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel"
# run_path = "/home/harishbabu/projects/PIPNet/runs/042-035_clone-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel"

# run_path = "/home/harishbabu/projects/PIPNet/runs/044-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20-or-4per-desc_orth-on-rel"

# run_path = "/home/harishbabu/projects/PIPNet/runs/046-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=10per-desc_orth-on-rel"
# run_path = "/home/harishbabu/projects/PIPNet/runs/047-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=5per-desc_tanh-desc"
# run_path = "/home/harishbabu/projects/PIPNet/runs/048-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=5per-desc_tanh-desc_unit-sphere"
# run_path = "/home/harishbabu/projects/PIPNet/runs/051-CUB-18-imgnet_cnext26_img=224_nprotos=4per-desc_tanh-desc_unit-sphere_AW=5-TW=2-UW=2-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/052-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=4per-desc_tanh-desc_unit-sphere_AW=5-TW=2-UW=2-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/055-CUB-18_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-softmax_AW=3-TW=2-UW=3-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/056-CUB-18-imgnet_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-softmax_AW=3-TW=2-UW=3-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/057-CUB-18-imgnet_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/058-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2"

# with unit sphere
# run_path = "/home/harishbabu/projects/PIPNet/runs/059-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_finetune=5_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# unit sphere with softmax
# run_path = "/home/harishbabu/projects/PIPNet/runs/065-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_finetune=5_no-meanpool_with-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# original hpipnet with 20 protos per node no KO, no OOD, no tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/062-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=20_no-KO_no-OOD"

# original hpipnet with 20 protos per node no KO, no OOD, WITH tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/063-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=20_no-KO_no-OOD_tanh-desc"

# with unit sphere but no AL+UNI
# run_path = "/home/harishbabu/projects/PIPNet/runs/066-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_finetune=5_no-meanpool_no-softmax_no-align_no-uni_AW=3-TW=2-UW=3-CW=2_batch=20"

# with unit sphere, protopool, with softmax, no tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/067-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# with unit sphere, protopool, with softmax, no tanh-desc, INCORRECT
# run_path = "/home/harishbabu/projects/PIPNet/runs/067-incorrect-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# with unit sphere, protopool, no softmax, no tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/068-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# 071 with bias
# run_path = "/home/harishbabu/projects/PIPNet/runs/071-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_with-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 072 gumbel softmax, tau=0.5
# run_path = "/home/harishbabu/projects/PIPNet/runs/072-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-gumbel-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 074 multiply_cs_softmax
run_path = "/home/harishbabu/projects/PIPNet/runs/074-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_multi-cs-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

try:
    sys.path.remove('/home/harishbabu/projects/PIPNet')
except:
    pass
sys.path.insert(0, os.path.join(run_path, 'source_clone'))


In [5]:
from pipnet.pipnet import PIPNet, get_network
from util.log import Log
from util.args import get_args, save_args, get_optimizer_nn
from util.data import get_dataloaders
from util.func import init_weights_xavier
from pipnet.train import train_pipnet, test_pipnet
# from pipnet.test import eval_pipnet, get_thresholds, eval_ood
from util.eval_cub_csv import eval_prototypes_cub_parts_csv, get_topk_cub, get_proto_patches_cub
from util.vis_pipnet import visualize, visualize_topk
from util.visualize_prediction import vis_pred, vis_pred_experiments
from util.node import Node
from util.phylo_utils import construct_phylo_tree, construct_discretized_phylo_tree
from util.func import get_patch_size
from util.data import ModifiedLabelLoader

Heatmaps showing where a prototype is found will not be generated because OpenCV is not installed.


In [6]:
from PIL import Image
import numpy as np
import pdb

def get_heatmap(latent_activation, input_image):
    image_a = latent_activation.cpu().numpy()
    image_a = (image_a - image_a.min()) / (image_a.max() - image_a.min())

    input_image = (input_image - input_image.min()) / (input_image.max() - input_image.min())
    image_b = input_image.permute(1, 2, 0).cpu().numpy()
    
    reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((input_image.shape[-1], input_image.shape[-1])))
    normalized_heatmap = (reshaped_image_a - np.min(reshaped_image_a)) / (np.max(reshaped_image_a) - np.min(reshaped_image_a))
    
    heatmap_colormap = plt.get_cmap('jet')
    heatmap_colored = heatmap_colormap(normalized_heatmap)
    
    heatmap_colored_uint8 = (heatmap_colored[:, :, :3] * 255).astype(np.uint8)
    image_a_heatmap_pillow = Image.fromarray(heatmap_colored_uint8)
    image_b_pillow = Image.fromarray((image_b * 255).astype('uint8'))
    
    result_image = Image.blend(image_b_pillow, image_a_heatmap_pillow, alpha=0.3)
    
    return np.array(result_image)


def get_heatmap_uninterpolated(latent_activation, input_image):
    image_a = latent_activation.cpu().numpy()
    image_a = (image_a - image_a.min()) / (image_a.max() - image_a.min())

    input_image = (input_image - input_image.min()) / (input_image.max() - input_image.min())
    image_b = input_image.permute(1, 2, 0).cpu().numpy()
    
    reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((input_image.shape[-1], input_image.shape[-1]), \
                                                                                          resample=Image.NEAREST ))
    normalized_heatmap = (reshaped_image_a - np.min(reshaped_image_a)) / (np.max(reshaped_image_a) - np.min(reshaped_image_a))
    
    heatmap_colormap = plt.get_cmap('jet')
    heatmap_colored = heatmap_colormap(normalized_heatmap)
    
    heatmap_colored_uint8 = (heatmap_colored[:, :, :3] * 255).astype(np.uint8)
    image_a_heatmap_pillow = Image.fromarray(heatmap_colored_uint8)
    image_b_pillow = Image.fromarray((image_b * 255).astype('uint8'))
    
    result_image = Image.blend(image_b_pillow, image_a_heatmap_pillow, alpha=0.3)
    ## Load Model
    return np.array(result_image)

## Load Model

In [7]:
args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
args = pickle.load(args_file)

if args.phylo_config:
    phylo_config = OmegaConf.load(args.phylo_config)

if args.phylo_config:
    # construct the phylo tree
    if phylo_config.phyloDistances_string == 'None':
        if '031' in run_path: # this run uses a different phylogeny file that had an extra root node which is a mistake
            root = construct_phylo_tree('/home/harishbabu/data/phlyogenyCUB/18Species-with-extra-root-node/1_tree-consensus-Hacket-18Species-modified_cub-names_v1.phy')
        else:
            root = construct_phylo_tree(phylo_config.phylogeny_path)
        print('-'*25 + ' No discretization ' + '-'*25)
    else:
        root = construct_discretized_phylo_tree(phylo_config.phylogeny_path, phylo_config.phyloDistances_string)
        print('-'*25 + ' Discretized ' + '-'*25)
else:
    # construct the tree (original hierarchy as described in the paper)
    root = Node("root")
    root.add_children(['animal','vehicle','everyday_object','weapon','scuba_diver'])
    root.add_children_to('animal',['non_primate','primate'])
    root.add_children_to('non_primate',['African_elephant','giant_panda','lion'])
    root.add_children_to('primate',['capuchin','gibbon','orangutan'])
    root.add_children_to('vehicle',['ambulance','pickup','sports_car'])
    root.add_children_to('everyday_object',['laptop','sandal','wine_bottle'])
    root.add_children_to('weapon',['assault_rifle','rifle'])
    # flat root
    # root.add_children(['scuba_diver','African_elephant','giant_panda','lion','capuchin','gibbon','orangutan','ambulance','pickup','sports_car','laptop','sandal','wine_bottle','assault_rifle','rifle'])
root.assign_all_descendents()

if ('num_protos_per_descendant' in args) and (args.num_protos_per_descendant > 0):
    for node in root.nodes_with_children():
        node.set_num_protos(args.num_protos_per_descendant)

------------------------- No discretization -------------------------


In [8]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    device_ids = [torch.cuda.current_device()]
else:
    device = torch.device('cpu')
    device_ids = []

args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
args = pickle.load(args_file)

# ckpt_file_name = 'net_overspecific_pruned_replaced_thresh=0.5_last'
ckpt_file_name = 'net_trained_last'
# ckpt_file_name = 'net_trained_10'
# ckpt_file_name = 'net_pretrained'
epoch = ckpt_file_name.split('_')[-1]

ckpt_path = os.path.join(run_path, 'checkpoints', ckpt_file_name)
checkpoint = torch.load(ckpt_path, map_location=device)

if ckpt_file_name != 'net_trained_last':
    print('\n', (10*'-')+'WARNING: Not using the final trained model'+(10*'-'), '\n')

# Obtain the dataset and dataloaders
trainloader, trainloader_pretraining, trainloader_normal, trainloader_normal_augment, projectloader, testloader, test_projectloader, classes = get_dataloaders(args, device)
if len(classes)<=20:
    if args.validation_size == 0.:
        print("Classes: ", testloader.dataset.class_to_idx, flush=True)
    else:
        print("Classes: ", str(classes), flush=True)

# Create a convolutional network based on arguments and add 1x1 conv layer
feature_net, add_on_layers, pool_layer, classification_layers, num_prototypes = get_network(len(classes), args, root=root)
   
# Create a PIP-Net
net = PIPNet(num_classes=len(classes),
                    num_prototypes=num_prototypes,
                    feature_net = feature_net,
                    args = args,
                    add_on_layers = add_on_layers,
                    pool_layer = pool_layer,
                    classification_layers = classification_layers,
                    num_parent_nodes = len(root.nodes_with_children()),
                    root = root
                    )
net = net.to(device=device)
net = nn.DataParallel(net, device_ids = device_ids)    
net.load_state_dict(checkpoint['model_state_dict'],strict=True)
net.eval()
criterion = nn.NLLLoss(reduction='mean').to(device)

# Forward one batch through the backbone to get the latent output size
# with torch.no_grad():
#     xs1, _, _ = next(iter(trainloader))
#     xs1 = xs1.to(device)
#     proto_features, _, _ = net(xs1)
#     wshape = proto_features['root'].shape[-1]
#     args.wshape = wshape #needed for calculating image patch size
#     print("Output shape: ", proto_features['root'].shape, flush=True)
    
args.wshape = 26

Dropping 0 samples from trainloader
Dropping 0 samples from trainloader_pretraining
Dropping 0 samples from trainloader_normal
Dropping 0 samples from trainloader_normal_augment
Num classes (k) =  18 ['cub_001_Black_footed_Albatross', 'cub_002_Laysan_Albatross', 'cub_003_Sooty_Albatross', 'cub_004_Groove_billed_Ani', 'cub_023_Brandt_Cormorant'] etc.
Classes:  {'cub_001_Black_footed_Albatross': 0, 'cub_002_Laysan_Albatross': 1, 'cub_003_Sooty_Albatross': 2, 'cub_004_Groove_billed_Ani': 3, 'cub_023_Brandt_Cormorant': 4, 'cub_024_Red_faced_Cormorant': 5, 'cub_025_Pelagic_Cormorant': 6, 'cub_031_Black_billed_Cuckoo': 7, 'cub_032_Mangrove_Cuckoo': 8, 'cub_033_Yellow_billed_Cuckoo': 9, 'cub_045_Northern_Fulmar': 10, 'cub_050_Eared_Grebe': 11, 'cub_051_Horned_Grebe': 12, 'cub_052_Pied_billed_Grebe': 13, 'cub_053_Western_Grebe': 14, 'cub_086_Pacific_Loon': 15, 'cub_100_Brown_Pelican': 16, 'cub_101_White_Pelican': 17}


/home/harishbabu/.conda/envs/hpnet1/lib/python3.8/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Number of prototypes:  20
----------Prototypes per descendant: 4----------
Assigned 136 protos to node root
Assigned 24 protos to node 052+053
Assigned 104 protos to node 004+086
Assigned 16 protos to node 053+050
Assigned 24 protos to node 004+032
Assigned 72 protos to node 086+045
Assigned 8 protos to node 050+051
Assigned 16 protos to node 032+033
Assigned 64 protos to node 045+101
Assigned 8 protos to node 033+031
Assigned 24 protos to node 045+003
Assigned 32 protos to node 101+023
Assigned 16 protos to node 003+002
Assigned 8 protos to node 101+100
Assigned 16 protos to node 023+025
Assigned 8 protos to node 002+001
Assigned 8 protos to node 025+024


In [28]:
root.get_node('004+086').descendents

from collections import Counter

# Assuming 'dataloader' is your DataLoader instance that has already been created
dataset = trainloader.dataset.dataset.dataset

# Get the mapping from class indices to names
# This assumes that your dataset has the 'classes' and 'class_to_idx' attributes
# which is the case for many built-in PyTorch datasets
idx_to_class = {v: k for k, v in dataset.class_to_idx.items()}

# Initialize a counter with class names
class_counts = Counter({class_name: 0 for class_name in dataset.classes})

# Iterate over the DataLoader
for *_, targets in trainloader:
    # Convert indices to class names
    targets = targets.numpy() if not isinstance(targets, list) else targets
    class_names = [idx_to_class[idx] for idx in targets]
    class_counts.update(class_names)

# Now class_counts has class names as keys and counts as values
print(class_counts)

Counter({'cub_001_Black_footed_Albatross': 30, 'cub_002_Laysan_Albatross': 30, 'cub_003_Sooty_Albatross': 30, 'cub_004_Groove_billed_Ani': 30, 'cub_023_Brandt_Cormorant': 30, 'cub_024_Red_faced_Cormorant': 30, 'cub_025_Pelagic_Cormorant': 30, 'cub_031_Black_billed_Cuckoo': 30, 'cub_032_Mangrove_Cuckoo': 30, 'cub_033_Yellow_billed_Cuckoo': 30, 'cub_045_Northern_Fulmar': 30, 'cub_050_Eared_Grebe': 30, 'cub_051_Horned_Grebe': 30, 'cub_052_Pied_billed_Grebe': 30, 'cub_053_Western_Grebe': 30, 'cub_086_Pacific_Loon': 30, 'cub_100_Brown_Pelican': 30, 'cub_101_White_Pelican': 30})


# ConvNext checks

In [ ]:
import torch
import torchvision

# Load the pre-trained ConvNeXt Tiny model
# model = torchvision.models.convnext_tiny(pretrained=True).cuda()
model = net.module._net

# Define a hook function that will print the shape of the inputs to each layer
def print_shape_hook(module, input, output):
    print(f"{module.__class__.__name__}: {output.shape}")

# Assuming you want to check the output after each stage
# The stages in the ConvNeXt Tiny model are contained in the 'features' module
for name, layer in model.named_children():
    if name == 'features':
        for idx, block in enumerate(layer):
            # Register hook to each block
            block.register_forward_hook(print_shape_hook)

# Turn off gradients as we are not doing training to save computation
with torch.no_grad():
    # Create a dummy input tensor of the correct size
    input_tensor = torch.randn(1, 3, 224, 224).cuda()
    
    # Get the model's output (forward pass) and triggers hooks
    output = model(input_tensor)

# Remove hooks after use if you're going to reuse this model and don't want the prints anymore
# This step is not necessary if the script ends after this since all objects will be deleted anyway


In [ ]:
import torch
import torchvision

# Load the pre-trained ConvNeXt Tiny model
model = torchvision.models.convnext_tiny(pretrained=True).cuda()
# model = net.module._net

# Define a hook function that will print the shape of the inputs to each layer
def print_shape_hook(module, input, output):
    print(f"{module.__class__.__name__}: {output.shape}")

# Assuming you want to check the output after each stage
# The stages in the ConvNeXt Tiny model are contained in the 'features' module
for name, layer in model.named_children():
    if name == 'features':
        for idx, block in enumerate(layer):
            # Register hook to each block
            block.register_forward_hook(print_shape_hook)

# Turn off gradients as we are not doing training to save computation
with torch.no_grad():
    # Create a dummy input tensor of the correct size
    input_tensor = torch.randn(1, 3, 224, 224).cuda()
    
    # Get the model's output (forward pass) and triggers hooks
    output = model(input_tensor)

# Remove hooks after use if you're going to reuse this model and don't want the prints anymore
# This step is not necessary if the script ends after this since all objects will be deleted anyway


In [9]:
for x in trainloader:
    pdb.set_trace()

> /tmp/ipykernel_128407/3201622953.py(1)<module>()
----> 1 for x in trainloader:
      2     pdb.set_trace()

ipdb> x
[tensor([[[[ 0.0056,  0.0056,  0.0056,  ...,  0.0056,  0.0056,  0.0056],
          [ 0.0056,  0.0056,  0.0056,  ...,  0.0056,  0.0056,  0.0056],
          [ 0.0056,  0.0056,  0.0056,  ...,  0.0056,  0.0056,  0.0056],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

         [[-0.0049, -0.0049, -0.0049,  ..., -0.0049, -0.0049, -0.0049],
          [-0.0049, -0.0049, -0.0049,  ..., -0.0049, -0.0049, -0.0049],
          [-0.0049, -0.0049, -0.0049,  ..., -0.0049, -0.0049, -0.0049],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0

ipdb> len(x)
3
ipdb> type(x[0])
<class 'torch.Tensor'>
ipdb> type(x[1])
<class 'torch.Tensor'>
ipdb> type(x[2])
<class 'torch.Tensor'>
ipdb> x[0].shape
torch.Size([20, 3, 224, 224])
ipdb> x[1].shape
torch.Size([20, 3, 224, 224])
ipdb> x[3].shape
*** IndexError: list index out of range
ipdb> x[2].shape
torch.Size([20])
ipdb> x[2]
tensor([13, 15, 13, 14, 12, 12,  8,  0,  0,  1,  9,  6, 12,  1,  5,  8, 16, 16,
        12,  8])
ipdb> q


# Weighted loss

In [15]:
import torch
import torch.nn.functional as F
outputs = torch.randn(10, 3, requires_grad=True)  # batch size of 10, 3 classes
labels = torch.randint(0, 3, (10,))  # batch size of 10

In [20]:
class WeightedCrossEntropyLoss(torch.nn.Module):
    def __init__(self):
        super(WeightedCrossEntropyLoss, self).__init__()

    def forward(self, inputs, targets, weights):
        print('inputs', inputs.shape)
        # Calculate cross entropy loss for each instance without reduction
        batch_loss = F.cross_entropy(inputs, targets, reduction='none')
        print('batch_loss', batch_loss.shape)
        
        # Create a mask based on the targets
        class_mask = F.one_hot(targets, num_classes=inputs.size(1)).type(inputs.dtype)
        
        print('class_mask', class_mask.shape)
        
        print('weights', weights.shape)
        
        # Multiply the loss by the weights vector
        weighted_losses = (batch_loss.unsqueeze(1) * class_mask * weights).sum(dim=1)
        
        print('weighted_losses', weighted_losses.shape)
        
        # Perform the final averaging over the batch
        loss = weighted_losses.mean()
        
        return loss

# Create an instance of the custom loss
custom_loss_function = WeightedCrossEntropyLoss()

# Example model output and target labels
# outputs = torch.randn(10, 3, requires_grad=True)  # batch size of 10, 3 classes
# labels = torch.randint(0, 3, (10,))  # batch size of 10

# Example weights for 3 classes
weights = torch.tensor([0.5, 2.0, 0.5], requires_grad=False)

# Calculate loss
loss = custom_loss_function(outputs, labels, weights)
print(loss)


inputs torch.Size([10, 3])
batch_loss torch.Size([10])
class_mask torch.Size([10, 3])
weights torch.Size([3])
weighted_losses torch.Size([10])
tensor(1.6013, grad_fn=<MeanBackward0>)


In [31]:
num_images_of_each_child = [60, 240]
weights = min(num_images_of_each_child) / torch.tensor(num_images_of_each_child, requires_grad=False)
print(weights)
print(weights.dtype)

tensor([1.0000, 0.2500])
torch.float32


# Freezing weights of dense layer

In [2]:
import torch
import torch.nn as nn

# Define a simple neural network with one dense layer
class MyNetwork(nn.Module):
    def __init__(self):
        super(MyNetwork, self).__init__()
        self.dense = nn.Linear(10, 5)  # Dense layer with 10 inputs and 5 outputs

    def forward(self, x):
        return self.dense(x)

# Instantiate the network
net = MyNetwork()

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Transfer the model to the chosen device (GPU or CPU if GPU is not available)
net.to(device)

# Freeze the first two weights of the dense layer
with torch.no_grad():
    for i in range(2):
        net.dense.weight[i].requires_grad = False

# Transfer the modified weights back to the device
net.dense.weight = net.dense.weight.to(device)

# You can check which parameters are trainable
for name, param in net.named_parameters():
    if param.requires_grad:
        print(f"Trainable: {name} on {param.device}")
    else:
        print(f"Non-trainable: {name} on {param.device}")

# Continue with your training process...


Using device: cuda
Trainable: dense.weight on cuda:0
Trainable: dense.bias on cuda:0


In [4]:
net.dense.weight[0].shape

torch.Size([10])

In [11]:
x = torch.ones((2, 6))
x[0, :0] = -1
x[0, 3:] = -1
x[1, :3] = -1
x[1, 6:] = -1

print(x)

tensor([[ 1.,  1.,  1., -1., -1., -1.],
        [-1., -1., -1.,  1.,  1.,  1.]])
